In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils.extmath import randomized_svd

from spectral_dagger.spectral.hankel import single_obs_basis, top_k_basis
from spectral_dagger.utils.math import normalize
from spectral_dagger.hmm import HMM
from spectral_dagger.spectral import SpectralPSR
from spectral_dagger.spectral import hankel
from spectral_dagger.envs import EgoGridWorld

In [ ]:
world = np.array([
    ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
    ['x', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'x', ' ', ' ', ' ', 'x'],
    ['x', ' ', 'G', 'x', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'x'],
    ['x', ' ', ' ', 'x', ' ', ' ', ' ', ' ', ' ', ' ', 'x', 'x', 'x', 'x'],
    ['x', ' ', ' ', 'x', 'S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'x'],
    ['x', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'x', ' ', ' ', ' ', 'x'],
    ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']]
)

n_colors = 1

env = EgoGridWorld(n_colors, world, noise=0.2)

# open loop policy
action_dist = normalize(np.ones(env.n_actions), ord=1)

T = np.tensordot(env.T, action_dist, ((0,), (0,)))
O = env.O[0, :, :]
O.squeeze()

init_dist = env.init_dist.copy()

n_states = T.shape[0]
n_obs = O.shape[1]

print n_states
print n_obs

observations = range(n_obs)
states = range(n_states)

hmm = HMM(observations, states, T, O, init_dist)

data_seed = np.random.randint(10000)
np.random.seed(data_seed)
print(hmm.sample_trajectory(10))

horizon = 10
n_samples = 1000
basis_max_length = 4
#dimension_seq = range(int(n_states/2.0), 2*n_states+1)
dimension_seq = range(20,40)

In [ ]:
def perplexity_score(psr, testing_data):
    return psr.get_perplexity(testing_data, base=2)

def make_prediction_score(hmm, horizon=2, n_samples=1000, ord=2):
    # Construct vector of probabilities at timestep t
    samples = [hmm.sample_trajectory(horizon) for i in range(n_samples)]
    probs = np.zeros(hmm.n_observations)
    
    for i, obs in enumerate(hmm.observations):
        for sample in samples:
            if sample[-1] == obs:
                probs[i] += 1.0 / n_samples
                
    assert np.isclose(sum(probs), 1.0)

    def prediction_score(psr, testing_data=None):
        psr_probs = np.array([
            psr.get_delayed_seq_prob([o], horizon-1) for o in hmm.observations])
             
        return np.linalg.norm(probs - psr_probs, ord=ord)
    
    return prediction_score

def fit_estimated_svd(
        hmm, training_data, dimension, estimator, basis_max_length):
    
    if estimator == 'single':
        basis = single_obs_basis(hmm.observations, True)
        estimator = 'substring'
    else:
        basis = top_k_basis(training_data, np.inf, estimator, basis_max_length)

    psr = SpectralPSR(hmm.observations)
    psr.fit(training_data, dimension, estimator, basis=basis)
    
    return psr
    
def fit_true_svd(
        hmm, training_data, dimension, estimator, basis_max_length):
    
    if estimator == 'single':
        basis = single_obs_basis(hmm.observations, True)
        estimator = 'substring'
    else:
        basis = top_k_basis(training_data, np.inf, estimator, basis_max_length)

    psr = SpectralPSR(hmm.observations)
    psr.fit(training_data, dimension, estimator, basis=basis)

    true_hankel = hankel.true_hankel_for_hmm(hmm, basis, horizon, estimator)

    n_oversamples = 10
    n_iter = 5

    #svd = randomized_svd(true_hankel, dimension, n_oversamples, n_iter)
    svd = np.linalg.svd(true_hankel, dimension)
    psr.fit(training_data, dimension, estimator, basis=basis, svd=svd)
    
    return psr


def scan_n_states(
        training_data, testing_data, hmm, dimension_seq,
        estimator, basis_max_length, score_func, fit_func):
    
    results = []
    
    for dim in dimension_seq:
        psr = fit_func(hmm, training_data, dim, estimator, basis_max_length)
        score = score_func(psr, testing_data)
        
        results.append((psr, dim, score))
    
    return min(results, key=lambda x: x[2])


In [ ]:
training_samples = [hmm.sample_trajectory(horizon) for i in range(n_samples)]
testing_samples = [hmm.sample_trajectory(horizon) for i in range(10)]

# score_func = perplexity_score
score_func = make_prediction_score(hmm)

# Do vanilla spectral learning
for e in ['string', 'prefix', 'substring', 'single']:
    model, n, perplexity = scan_n_states(
        training_samples, testing_samples, hmm, dimension_seq, 
        e, basis_max_length, score_func, fit_estimated_svd)
    
    print "^" * 40
    print "Estimated SVD with estimator %s" % e
    print "Dimension: ", n
    print "Perplexity: ", perplexity
    print "^" * 40
    
# Do spectral learning with true U, S, V
for e in ['string', 'prefix', 'substring', 'single']:
    model, n, perplexity = scan_n_states(
        training_samples, testing_samples, hmm, dimension_seq, 
        e, basis_max_length, score_func, fit_true_svd)
    
    print "@" * 40
    print "True SVD with estimator %s" % e
    print "Dimension: ", n
    print "Perplexity: ", perplexity
    print "@" * 40